# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from IPython.display import display

#### Using **event_datafile_new.csv** directly

In [2]:
# Using event_datafile_new.csv File directly due to incorrect data in "event_data" directory.

file = 'event_datafile_new.csv'
event_data = pd.read_csv(os.getcwd()+"/"+file) 
display(event_data.iloc[0:2])

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Miami Horror,Kate,F,88,Harrell,250.8273,paid,"Lansing-East Lansing, MI",293,Sometimes,97
1,The White Stripes,Kate,F,89,Harrell,241.8673,paid,"Lansing-East Lansing, MI",293,My Doorbell (Album Version),97


In [3]:
# checking the number of rows in csv file
data_rows_list = []
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print("Rows in File = "+str(sum(1 for line in f)))
print("DataFrame Rows = "+str(event_data.shape[0]))
print("DataFrame Columns = "+str(event_data.shape[1]))


Rows in File = 6821
DataFrame Rows = 6820
DataFrame Columns = 11


----
# Part II. Apache Cassandra, CQL part of the project. 

## Ingesting CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory to the following three tables using Pandas DataFrame.  
## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra code

#### Creating a Cluster

In [4]:
# Connecting to a Cassandra instance on the local machine 

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect() # To establish connection and begin executing queries, need a session
except Exception as e:
    print(e)


#### Creating Keyspace

In [5]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_keyspace 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [6]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('music_keyspace')
except Exception as e:
    print(e)

----
### Creating queries to answer the following three questions of the data:

##### 1. Fetching the artist, song title and song's length in the music app history that was heard during sessionId = 338 and itemInSession  = 4

##### 2. Fetching the name of artist, song (sorted by itemInSession) and user's first and last name for userid = 10 and sessionid = 182
    
##### 3. Fetching the user name (first and last) in the music app history who listened to the song 'All Hands Against His Own'
</br>

### **Query-1**

In [7]:
## Query 1:  Fetching the artist, song title and song's length in the music app history
## Condition - sessionId = 338, and itemInSession = 4
drop_session_details = "DROP TABLE IF EXISTS session_details" 

create_session_details = """
CREATE TABLE IF NOT EXISTS session_details 
(
    session_id INT, 
    item_in_session INT,
    artist_name TEXT, 
    song_name TEXT, 
    song_length DECIMAL, 
    PRIMARY KEY (
        session_id, 
        item_in_session
    )
)
"""
try:
    session.execute(drop_session_details)
    session.execute(create_session_details)
except Exception as e:
    print(e)                   

In [8]:
#INSERT statement for table session_details
insert_session_details = """
INSERT INTO session_details
(session_id, item_in_session, artist_name, song_name, song_length)
VALUES (%s, %s, %s, %s, %s)
"""
for index, row in event_data.iterrows():
    data_to_insert = (row.sessionId, row.itemInSession, row.artist, row.song, row.length)
    session.execute(insert_session_details, data_to_insert)


In [9]:
## Fetching the output with conditions sessionId = 338 and itemInSession = 4
query = """
SELECT 
    session_id, 
    item_in_session, 
    artist_name, 
    song_name, 
    song_length 
FROM 
    session_details 
WHERE 
    session_id = 338 
    AND item_in_session  = 4
"""
try:
    out_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)
display(out_df)

,session_id,item_in_session,artist_name,song_name,song_length
0,338,4,Faithless,Music Matters (Mark Knight Dub),495.3073


### **Query-2**

In [10]:
## Query 2: Fetching name of artist, song  and user's first and last name
## Condition -  userid = 10, sessionid = 182
## Order - sorted by itemInSession
drop_user_session_details = "DROP TABLE IF EXISTS user_session_details" 
create_user_session_details = """
CREATE TABLE IF NOT EXISTS user_session_details 
(
    user_id INT, 
    session_id INT, 
    item_in_session INT, 
    artist_name TEXT, 
    song_name TEXT, 
    firstname TEXT,
    lastname TEXT, 
    PRIMARY KEY 
    (
        (
            user_id,
            session_id
        ),
        item_in_session
    )
)
"""
try:
    session.execute(drop_user_session_details)
    session.execute(create_user_session_details)
except Exception as e:
    print(e)           

In [11]:
#INSERT statement for table session_details
insert_session_details = """
INSERT INTO user_session_details
(user_id, session_id, item_in_session, artist_name, song_name, firstname, lastname)
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
for index, row in event_data.iterrows():
    data_to_insert = (row.userId, row.sessionId, row.itemInSession, row.artist, row.song, row.firstName, row.lastName)
    session.execute(insert_session_details, data_to_insert)


In [12]:
query = """
SELECT 
    user_id, 
    session_id, 
    item_in_session,
    artist_name, 
    song_name, 
    firstname, 
    lastname
FROM 
    user_session_details 
WHERE 
    user_id = 10
    AND session_id = 182
"""
try:
    out_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)
display(out_df)

,user_id,session_id,item_in_session,artist_name,song_name,firstname,lastname
0,10,182,0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,10,182,1,Three Drives,Greece 2000,Sylvie,Cruz
2,10,182,2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,10,182,3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


### **Query-3**

In [13]:
## Query 3: Fetching user's first and last name in my music app history 
## Condition - Listened to the song 'All Hands Against His Own'
drop_user_activity = "DROP TABLE IF EXISTS user_activity" 
create_user_activity = """
CREATE TABLE IF NOT EXISTS user_activity 
(
    song_name TEXT, 
    user_id INT, 
    firstname TEXT,
    lastname TEXT, 
    PRIMARY KEY 
    (
        (
            song_name
        ),
        user_id
    )
)
"""
try:
    session.execute(drop_user_activity)
    session.execute(create_user_activity)
except Exception as e:
    print(e)          

In [14]:
insert_user_activity = """
INSERT INTO user_activity
(song_name, user_id, firstname, lastname)
VALUES (%s, %s, %s, %s)
"""
for index, row in event_data.iterrows():
    data_to_insert = (row.song, row.userId, row.firstName, row.lastName)
    session.execute(insert_user_activity, data_to_insert)

In [15]:
query = """
SELECT 
    song_name, 
    firstname, 
    lastname
FROM 
    user_activity 
WHERE 
    song_name = 'All Hands Against His Own'
"""
try:
    out_df = pd.DataFrame(list(session.execute(query)))
except Exception as e:
    print(e)
display(out_df)

,song_name,firstname,lastname
0,All Hands Against His Own,Jacqueline,Lynch
1,All Hands Against His Own,Tegan,Levine
2,All Hands Against His Own,Sara,Johnson


### Dropping the tables before closing out the sessions

In [16]:
## Dropping the table before closing out the sessions
try:
    session.execute(drop_session_details)
    session.execute(drop_user_session_details)
    session.execute(drop_user_activity)
except Exception as e:
    print(e)          

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()

## Notebook Ends
-----